# Introduction

This notebook details the ensemble model consisting of three separate  deep neural network models. The model weights are loaded and averaged in this notebook. Details and code for the separate models can be found here:
1. [Model 1](https://github.com/AmyRouillard/DSI-FCANS/blob/development/notebooks/dnn-base-model-1.ipynb)
2. [Model 2](https://github.com/AmyRouillard/DSI-FCANS/blob/development/notebooks/model-2-10fold-model-2.ipynb)
3. [Model 3](https://github.com/AmyRouillard/DSI-FCANS/blob/development/notebooks/model-3-10fold.ipynb)

This notebook is made up of the following sections:

1. Importing libraries
2. Data importation
3. Data wrangling
4. Utility functions
5. Model architecture and Load Weights
6. Model Ensemble
7. Api Submission


## 1. Importing Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

## 2. Data Importation

In [ ]:
# Track time to load dataset
!%%time

# Declare number of ananonymized features
n_features = 300

# Select anonymized features
features = [f'f_{i}' for i in range(n_features)]

# Import train set
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')

## 3. Data Wrangling

This section handles:

* Selecting the independent and dependent features from the data set.
* Dropping time_id feature (will not be utilized in modeling).
* Create an integer look up layer for investment _id feature.

In [ ]:
investment_id = train.pop("investment_id")
investment_id.head()

In [ ]:
_ = train.pop("time_id")

In [ ]:
y = train.pop("target")
y.head()

## 3.1 Create an IntegerLookup layer for investment_id input

In [ ]:
%%time
investment_ids = list(investment_id.unique())
investment_id_size = len(investment_ids) + 1
investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
with tf.device("cpu"):
    investment_id_lookup_layer.adapt(investment_id)

## 4. Utility Functions

In [ ]:
# Making Tesorflow dataset
import tensorflow as tf
def preprocess(X, y):
    
    """
    .Pre-processing a tensorflow dataset
    
    Parameters
    ----------
    X : array, a list of features

    y : array, a feature
    
    
    """
    return X, y


def make_dataset(feature, investment_id, y, batch_size=1024, mode="train"):
    
    """ Function to create a source dataset compatable with tensorflow. 
    In addition a dataset transformation is applied  and the data is shuffled 
    if it is part of the training set. 

    Parameters
    ----------
    feature : array, shape = [n, 300]
        300 annonymised features.
    investment_id : list of int, shape = [n]
        List of investment Ids.
    y : array, shape = [n]
        Array containing target values we wish to predict.
    batch_size : int, default = 1024
        Size of batches.
    mode : string, default = "train"
        Variable used to specify if the data if from the training, test or
        validation data sets.
    
    Returns
    -------
    ds : tensorflow dataset, class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset' 
        Dataset in format compatible for training model.
    
    """
    
    ## Read elements from memory
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature), y))
    
    ## Map preprocess function
    ds = ds.map(preprocess)
    
    ## If mode is set to train shuffle data
    if mode == "train":
        ds = ds.shuffle(256)
        
    # Combine consecutive elements of this dataset into batches.
    # Cache the elements in dataset
    # Allow later elements to be prepared while the current element is being processed (prefetch)
    
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    
    return ds

## 5. Model architecture and Load Weights

In [ ]:
# Define the three models used for the ensemble learning

def get_model():   
    """ 
    
    Function to define the multi-input keras model 1 architecture. 

    Returns
    -------
    
    model : model, class 'keras.engine.functional.Functional'
        Model groups layers into an object with training and inference features.
    
    """
    
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model


def get_model2():
    """ 
    
    Function to define the multi-input keras model 2 architecture. 

    Returns
    -------
    
    model : model, class 'keras.engine.functional.Functional'
        Model groups layers into an object with training and inference features.
    
    """
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)    
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
  
   
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.65)(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
   # x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
  #  x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.75)(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model


def get_model3():
    """ 
    
    Function to define the multi-input keras model 3 architecture. 

    Returns
    -------
    
    model : model, class 'keras.engine.functional.Functional'
        Model groups layers into an object with training and inference features.
    
    """
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float32)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    investment_id_x = layers.Dense(32, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(128, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(64, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1)(x)
    output = tf.keras.layers.BatchNormalization(axis=1)(output)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model


In [ ]:
# save memory
del train
del investment_id
del y
gc.collect()

In [ ]:
%%capture

#initialize models list and load weights of the models trained outside this notebook

models = []
for i in range(5):
    model = get_model()
    model.load_weights(f'../input/base-model-dnn/model_{i}')
    models.append(model)

for i in range(10):
    model = get_model2()
    model.load_weights(f'../input/model-2-10fold/model_{i}')
    models.append(model)
    
    
for i in range(10):
    model = get_model3()
    model.load_weights(f'../input/model-3-10fold/model_{i}')
    models.append(model)

## 6. Model Ensemble

In [ ]:
def preprocess_test(investment_id, feature):
    return (investment_id, feature), 0

def make_test_dataset(feature, investment_id, batch_size=1024):
    """ Function to create a source dataset from the test features compatable 
    with tensorflow. 
    In addition a dataset transformation is applied  and the data is shuffled 
    if it is part of the training set.

    Parameters
    ----------
    feature : array, shape = [n, 300]
        Ground truth (correct) target values.
    investment_id : list of int, shape = [n]
        List of investment Ids.
    batch_size : int, default = 1024
        Size of batches.
    
    Returns
    -------
    ds : tensorflow dataset, class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset' 
        Dataset in format compatible for training model.
        .
    """
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def inference(models, ds):
    """ Make predictions unsing n models in models and return mean of predictions.
    Parameters
    ----------
    models : array like, shape = [n]
        Trained models.
    ds : tensorflow dataset, class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset' 
        Dataset in format compatible for training model.
    Returns
    -------
    mean_y_pred : float
        Mean values of preditions made my each model in models.
    
    """
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
    return np.mean(y_preds, axis=0)

## 7. API Submission

Finaly we call kagle's API for test data and make predictions


In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test() 
for (test_df, sample_prediction_df) in iter_test:
    ds = make_test_dataset(test_df[features], test_df["investment_id"])
    sample_prediction_df['target'] = inference(models, ds)
    env.predict(sample_prediction_df) 
    display(sample_prediction_df)